In [1]:
# import matplotlib
# %matplotlib WebAgg
%matplotlib qt
import os
root = os.path.realpath(os.getcwd()+ os.sep+"..")
import sys
sys.path.append(root)
from modules.WiFi import WiFi, Modes, make_drive_command
# import pathfinder, pathfinder_visualizer
import captain
import importlib
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import time
import re, json
from datetime import datetime
import threading

In [2]:
# show_visual()

from ipywidgets import Button, VBox, ToggleButton, IntSlider, HBox, HTML, Layout, Output

start_pathfinder = Button(
    description="Start Captain",
    layout = Layout(width='auto', height='80px', margin='10px'), 
    style=dict(font_size="60px"),
    button_style="primary",
    disabled = captain._service.is_running())
stop_pathfinder = Button(
    description="Stop Captain", 
    layout= start_pathfinder.layout, 
    style=start_pathfinder.style,
    button_style="warning",
    disabled = not captain._service.is_running())
run_rover = ToggleButton(
    value=False, 
    description="Run Rover", 
    layout= Layout(flex='1', height='80px', margin='10px'), 
    style=dict(font_size="60px"),
    button_style="success",
    disabled = not captain._service.is_running())
visual_button = Button(
    description="PLT", 
    layout = Layout(height='auto', margin='10px'))
speed_slide = IntSlider(
    value=0, min=0, max=100, 
    orientation="vertical", 
    layout=Layout(height="auto"))
# text_display = HTML(
#     value="Some HTML text", 
#     # layout= Layout(flex='1', margin="10px", border="solid", padding="10px"))
#     layout= )
text_output = Output(layout=Layout(flex='1', max_height="500px", min_height="100px"))
poll_rover = Button(
    description="Poll Rover",
    layout = Layout(flex='1')
)
clear_output = Button(
    description="Clear Output"
)

buttons = VBox([
    start_pathfinder,
    HBox([run_rover,], layout=Layout(width='auto', padding='0px', margin='0px')),
    stop_pathfinder,
    ],
    layout = Layout(flex='1')
)
controls = HBox([
    speed_slide, buttons, speed_slide
])
logging = HBox([
    HBox([text_output], layout=Layout(border='solid', flex='1', padding='10px')), VBox([poll_rover, clear_output])
    ],
    layout = Layout(margin='10px')
)
gui = VBox([
    controls,
    logging
])

Starting Captain Service
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Stop Captain Service
Starting Captain Service
Stop Captain Service
Starting Captain Service
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Stop Captain Service
Starting Captain Service
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Stop Captain Service
Starting Captain Service
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Stop Captain Service
Starting Captain Service
Stop Captain Service
Starting Captain Service
Stop Captain Service
Starting Captain Service
Stop Captain Service
Starting Captain Service
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Stop Captain Service
Starting Captain Service
Stop Captain Service
Starting Captain Service
Stop Captain Service
Starting Captain Service
Stop Captain Service
Starting 

In [3]:
def update_button_disables(pathfinding):
    start_pathfinder.disabled = pathfinding
    run_rover.disabled = not pathfinding
    stop_pathfinder.disabled = not pathfinding

def on_start_pathfinding(b):
    importlib.reload(captain)
    def command_printer(i):
        with(open("Command.txt", "w")) as f:
            f.write(f"{datetime.now().strftime('%H:%M:S')} => {i}")
    captain.start_captain_service(lambda: speed_slide.value, print)
    update_button_disables(True)
start_pathfinder.on_click(on_start_pathfinding)
def on_stop_pathfinding(b):
    captain.stop_captain_service()
    update_button_disables(False)
stop_pathfinder.on_click(on_stop_pathfinding)

def rover_order_changed(change):
    run = change['new']
    if run:
        run_rover.button_style='danger'
        run_rover.description='Stop Rover'
        captain.config['send_commands_to_rover'] = True
    else:
        captain.captain_stop()
        captain.config['send_commands_to_rover'] = False
        run_rover.button_style='success'
        run_rover.description='Run Rover'
run_rover.observe(rover_order_changed, 'value')
def on_poll_rover(b):
    with text_output:
        stat = captain.rover.get_status(1, 0)
        print("No status received") if stat is None else print({k:stat[k] for k in stat if re.match("^current.*", k)})
poll_rover.on_click(on_poll_rover)
clear_output.on_click(lambda b: text_output.clear_output())

In [4]:
# pathfinder_visualizer.show_visual(pathfinder)

In [5]:
gui

In [6]:
captain.show_captain_visual()

In [7]:
# rover.send_command(make_command(Modes.SPIN,speed_percent=0))

In [ ]:
# rover.get_status()

called start_path_finder_service
Starting Pathfinder Service
Called start_worldview_service
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Starting Worldview Service
Called start_lidar_service
Starting Lidar Service
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, -10]}
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, -11]}
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, -11]}
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, -11]}
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, -11]}
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, -11]}
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, -11]}
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, -11]}
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, 52]}
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, 59]}
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, 60]}
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, 60]}
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, 38]}
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 14]}

Too many bytes in the input buffer: 3251/3000. Cleaning buffer...


Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [13, -11]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [13, -11]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [13, -11]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [13, -11]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [-13, -33]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [-13, -35]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [13, -14]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [-13, -33]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [-13, -35]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [-13, -36]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [-13, -36]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [13, 7]}
Data sent successfully
{'HB': 0, 'IO

Too many bytes in the input buffer: 3036/3000. Cleaning buffer...


Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [-13, -60]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [-13, -60]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [-13, -60]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [-13, -60]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [-13, -60]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [-13, -60]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [-13, -60]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [-13, -60]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [-13, -60]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [-13, -60]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [-13, -60]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [-13, -60]}
Data sent successfully
{'HB'

...

Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}

Too many bytes in the input buffer: 4715/3000. Cleaning buffer...


Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -31]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -57]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -38]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -36]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -57]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -59]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -60]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -60]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -38]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -36]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -36]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM

Too many bytes in the input buffer: 3042/3000. Cleaning buffer...


Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Data sent successfully
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}